In [2]:
# load the file
import sys
import io
import os

fileDir = "../Data/canopies_clean/"
listfiles = os.listdir(fileDir)
allPapersInFile = []

for file in listfiles:
    if not file.startswith('.'):
        with open(fileDir+file, 'r', encoding = 'utf8') as f:
            for line in f:
                read_data = line.split("\t")
                paper_detail = {"paperID": read_data[0], "authorID":read_data[1],
                                "author_position":read_data[2], "total_author": read_data[3],
                                "author_name": read_data[4], "co_authors": read_data[5], 
                                "department": read_data[6], "vendor": read_data[7], 
                                "mesh": read_data[8], "keywords": read_data[9], 
                                "publish_year": read_data[10], "doi": read_data[11],
                                "paperTitle": read_data[12]}
                allPapersInFile.append(paper_detail)
        f.close()
print("Total:",len(allPapersInFile), "papers")

Total: 140266 papers


In [9]:
# extract abstract based on paperID and generate new smaller file with id to abstract
# sort paperID
paperIDs = []
for paper in allPapersInFile:
    paperIDs.append(int(paper["paperID"]))
paperIDs.sort()
# extract abstract based on paper id
filePath = "../Data/id_abstract.txt"
with open(filePath, 'r', encoding = 'utf8') as f:
    #newf = open("../Data/id_abstract_processed.txt", "w+")
    for i, line in enumerate(f):
        if (i%1000==0):
            print(i)
        for pid in paperIDs:
            if (i==pid):
                read_data = line.split("\t")
                print(pid)
                paperIDs.remove(pid)
                print(read_data[0]+"\t"+read_data[1])
                break;
    #newf.close()
f.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
8077
8630	A series of N1-p-chlorophenyl-N5-alkylbiguanides were synthesized as potential inhibitors of dental plaque. Partition coefficients and pKa values were determined by standard methods. Biological activity was evaluated against Streptococcus mutans, a pure strain of plaque-forming bacteria. All compounds were compared to chlorhexidine acetate.



KeyboardInterrupt: 

In [3]:
# data column
for key in allPapersInFile[0]:
    print(key)

paperID
authorID
author_position
total_author
author_name
co_authors
department
vendor
mesh
keywords
publish_year
doi
paperTitle


In [4]:
from collections import Counter

# create author ID to name map
author_ID_to_name_map = []

for paper in allPapersInFile:
    if(paper["authorID"] not in [item["author_ID"] for item in author_ID_to_name_map]):
        author_ID_to_name_map.append({"author_name": paper["author_name"], "author_ID": paper["authorID"]})

# extract author with same name
sameNameAuthor = []
for i in author_ID_to_name_map:
    for j in author_ID_to_name_map:
        if(j["author_name"]==i["author_name"] and j["author_ID"]!=i["author_ID"]):
            if(i["author_ID"] not in [item["author_ID"] for item in sameNameAuthor]):
                sameNameAuthor.append(i)
            if(j["author_ID"] not in [item["author_ID"] for item in sameNameAuthor]):
                sameNameAuthor.append(j)

# # count author with same name
# sameNameCounter = Counter([author["author_name"] for author in sameNameAuthor])
# for key, value in sameNameCounter.items():
#     if value>=2:
#         print(value, "people have same name:", key)

# for author in sameNameAuthor:
#     print("Author: {k}, Author ID: {c}".format(k=author["author_name"], c=author["author_ID"]))    

In [12]:
# filter out authors that have same name but write less paper than threshold

# set up threshold
threshold = 10

# count number of paper author write
c = Counter([paper["authorID"] for paper in allPapersInFile])
for key, value in c.items():
    for author in sameNameAuthor:
        if(key == author["author_ID"]):
            author["paper_count"] = value

sameNameAuthorWithCount = sorted(sameNameAuthor, key=lambda k: (k['author_name'], k["paper_count"]))
# collect authors that have repeated name and write more paper than threshold
uniqueAuthors = []
temp = []
authorFilter = []
for author in sameNameAuthorWithCount:
    if(author["paper_count"]>threshold):
        temp.append(author)
        #print("Author name: {n}, Author: {k}, Paper count: {c}".format(n=author["author_name"],k=author["author_ID"], c= author["paper_count"]))

for author in temp:
    if(author["author_name"] not in uniqueAuthors):
        uniqueAuthors.append(author["author_name"])
    else: 
        if author["author_name"] not in authorFilter:
            authorFilter.append(author["author_name"])
# collect filtered data
filteredAuthors = []
for author in temp:
    if(author["author_name"] in authorFilter):
        filteredAuthors.append(author)

for author in filteredAuthors:
    print("Author name: {n}, Author: {k}, Paper count: {c}".format(n=author["author_name"],k=author["author_ID"], c= author["paper_count"]))


Author name: a patel, Author: 0000-0003-3874-3216, Paper count: 11
Author name: a patel, Author: 0000-0001-7214-5901, Paper count: 18
Author name: ana castro, Author: 0000-0003-4035-3444, Paper count: 11
Author name: ana castro, Author: 0000-0001-6964-6879, Paper count: 13
Author name: ana castro, Author: 0000-0001-7526-6717, Paper count: 39
Author name: anna ferrari, Author: 0000-0002-7022-9906, Paper count: 17
Author name: anna ferrari, Author: 0000-0001-9536-3995, Paper count: 49
Author name: bharat bhushan, Author: 0000-0002-4182-9478, Paper count: 19
Author name: bharat bhushan, Author: 0000-0001-7161-6601, Paper count: 163
Author name: bin liu, Author: 0000-0002-5836-2333, Paper count: 14
Author name: bin liu, Author: 0000-0002-0956-2777, Paper count: 97
Author name: carmen moreno, Author: 0000-0002-1660-7072, Paper count: 13
Author name: carmen moreno, Author: 0000-0003-0541-4846, Paper count: 35
Author name: carmen torres, Author: 0000-0001-8573-0990, Paper count: 20
Author nam